[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [3]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Set the environment variable if it exists in .env
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY: ")

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [6]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

llama32_chat = ChatOllama(model="llama3.2:latest", temperature=0)
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

C:\Users\YT40432\AppData\Local\Temp\ipykernel_7068\2577417745.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llama32_chat = ChatOllama(model="llama3.2:latest", temperature=0)


Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [7]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
llama32_chat.invoke(messages)

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-02-25T14:58:26.7048433Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1685221300, 'load_duration': 1341335600, 'prompt_eval_count': 27, 'prompt_eval_duration': 106000000, 'eval_count': 25, 'eval_duration': 236000000}, id='run-e95e25ba-2b3b-4777-b232-d9ea1d21981f-0')

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
llama32_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-02-25T14:40:56.5482447Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 460787700, 'load_duration': 17861900, 'prompt_eval_count': 27, 'prompt_eval_duration': 4000000, 'eval_count': 10, 'eval_duration': 432000000}, id='run-c748b85b-1aac-468b-b141-ba627a564afb-0')

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [8]:
import os
from dotenv import load_dotenv

# Define the path to your .env file
env_path = r"c:\Users\YT40432\Desktop\lp\research\lucaspecina\langchain-academy\.env"

# Check if the file exists
print(f"File exists: {os.path.exists(env_path)}")

# Read the file content (without displaying the full keys for security)
try:
    with open(env_path, 'r') as f:
        content = f.read()
        lines = content.strip().split('\n')
        for line in lines:
            if '=' in line:
                key, value = line.split('=', 1)
                # Show first few and last few characters of the value
                if len(value) > 10:
                    masked_value = value[:5] + '...' + value[-5:]
                else:
                    masked_value = value
                print(f"{key} found with value starting with {masked_value}")
except Exception as e:
    print(f"Error reading .env file: {e}")

# Try loading the .env file
load_dotenv(dotenv_path=env_path)

# Check if the API keys were loaded into environment variables
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print(f"OPENAI_API_KEY loaded into environment: {'Yes' if openai_key else 'No'}")
if openai_key:
    print(f"OPENAI_API_KEY starts with: {openai_key[:5]}... and ends with: ...{openai_key[-5:]}")
    
print(f"TAVILY_API_KEY loaded into environment: {'Yes' if tavily_key else 'No'}")
if tavily_key:
    print(f"TAVILY_API_KEY starts with: {tavily_key[:5]}... and ends with: ...{tavily_key[-5:]}")

# Check if the keys look valid
if openai_key and "None" in openai_key:
    print("WARNING: Your OPENAI_API_KEY appears to contain 'None', which suggests it may not be valid")
    
if tavily_key and not tavily_key.startswith("tvly-"):
    print("WARNING: Your TAVILY_API_KEY doesn't start with 'tvly-', which suggests it may not be valid")

File exists: True
OPENAI_API_KEY found with value starting with sk-No...gsVcz
TAVILY_API_KEY found with value starting with tvly-...5B6Fm
OPENAI_API_KEY loaded into environment: Yes
OPENAI_API_KEY starts with: sk-No... and ends with: ...gsVcz
TAVILY_API_KEY loaded into environment: Yes
TAVILY_API_KEY starts with: tvly-... and ends with: ...5B6Fm


In [9]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
# Since .env is one directory up from the current file
load_dotenv(dotenv_path="../.env")

# Get the Tavily API key from environment variables
tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [10]:
tavily_api_key

'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'

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=3)

search_docs = tavily_search.invoke("What is LangGraph?")

In [12]:
search_docs

[{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.'},
 {'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
  'content': 'In this article, we’ll introduce LangGraph, walk you through it

In [18]:
for i in range(len(search_docs)):
    print('---------------')
    for k, v in search_docs[i].items():
        print(k)
        print('     ', v)

---------------
url
      https://www.datacamp.com/tutorial/langgraph-tutorial
content
      LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactions.
---------------
url
      https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141
content
      In this article, we’ll introduce LangG